In [22]:
#2/21/17 Chris Burns

#Imports
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline


#manipulation parameters
gauss_kernel = 3
canny_low = 50
canny_high = 150
rho = 1
theta = np.pi/180
hough_threshold = 1
minlen = 35
maxgap = 10

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img):
    return cv2.Canny(img, canny_low, canny_high)

def gaussian_blur(img):
    return cv2.GaussianBlur(img,(gauss_kernel, gauss_kernel), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=5):

    right_lines = []
    left_lines = []
    
    for l in range(len(lines)):
        line = lines[l][0]
        #Calculate slope of the lines
        #slope = ((y2-y1)/(x2-x1))
        slope = (line[3] - line[1])/(line[2] - line[0])
        if slope >= 0.195:
            right_lines.append(line)
        elif slope <= -0.195:
                left_lines.append(line)
    
    right_left = [right_lines, left_lines]
            
    # Fit and draw
    for new_lines in right_left:
        final_annotation = np.array(extrapolate(new_lines, img),np.int32)
        if len(final_annotation) >= 0:
            cv2.line(img, (final_annotation[0][0], final_annotation[0][1]), (final_annotation[1][0],final_annotation[1][1]), color, thickness= 10)



def extrapolate(lines, img):
    
    if(len(lines)==0):
        return np.zeros ((2,2))
    
    #seperate X and Y by Right and Left
    x_list = []
    y_list = []
    for line in lines:
        x_list.append(line[0])
        x_list.append(line[2])
        y_list.append(line[1])
        y_list.append(line[3])
    
    # Fit
    line = np.polyfit(y_list,x_list, 1)
    f = np.poly1d(line)
    
    y = np.linspace(img.shape[0],340, 2)
    x = f(y)
   
    return list(zip(x,y))


def hough_lines(img):

    lines = cv2.HoughLinesP(img, rho, theta, hough_threshold, np.array([]), minLineLength=minlen, maxLineGap=maxgap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img


def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    return cv2.addWeighted(initial_img, α, img, β, λ)


#Primary pipeline for processing an image through the necessary steps
def process_image(image):
    
    #read the image in from Disk
    origImage = np.copy(image)
    
    # 1.) Convert to grayscale for (presumably) better contrast
    grayed_image = grayscale(image)
   
    #2.) Add a gaussian blur to further contrast the lane lines with the environment
    gaussed_image = gaussian_blur(image)

    #3.) Apply a Canny Filter
    canny_image = canny(gaussed_image)

    #4.) Apply ROI Mask
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(450, 340), (490, 300), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_image = region_of_interest(canny_image, vertices)
    
    #5.) Hough Transform
    hough_image = hough_lines(masked_image)
    

    #6.) Draw our Lines on the image
    processed_image = weighted_img(hough_image,origImage)
    
    return processed_image


#Save processed images to the same directory they came from 
#Append xf_ to the filename (Xf = "Transformed File")
def save_xf_image(org_filename,image):
    
    newFileName = "test_images/xf_{}".format(org_filename)
    plt.imsave(newFileName, image)
    
    plt.imshow(image)
    plt.show()
    
    
#Feed the pipeline with all the images in a directory
"""
image_files = os.listdir("test_images/")
for file in image_files:
    print("processing: {}".format(file))
     #read the image in from Disk
    image = mpimg.imread("test_images/{}".format(file))
    #process the image file
    xfImage = process_image(image)
    
    #display and save the image_file
    save_xf_image(file, xfImage)
"""


white_output = 'lanelineannotation.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio = False)

HTML("""
<video width="960" height="540" controls>
    <source src="{0}">
</video>
""".format(white_output))




[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:09<00:00, 21.60it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 5.63 s, sys: 480 ms, total: 6.11 s
Wall time: 10.9 s
